In [ ]:
import os
from zipfile import ZipFile
from google.colab import files

# delete any garbage in the environment
os.system("rm -r *")

#@markdown ### Upload a zip file containing the pdb files you want to cluster.
#@markdown **Note:**
#@markdown - The zip file cannot contain any files that aren't valid PDB or PDBx/mmCIF files.
#@markdown - The zip file cannot have any sub directories or folders within it.
#@markdown - The upload will only use the first file upload, consequent files will be ignored.
uploaded = files.upload()
filename = list(uploaded.keys())[0]

os.makedirs("pdbs/", exist_ok=True)

# Extract all the contents of zip file in current directory
with ZipFile(filename) as zf:
   zf.extractall("pdbs/")

# check if pdbs in sub directory
files = os.listdir("pdbs")
if len(files) == 1 and os.path.isdir(f"pdbs/{files[0]}"):
  os.system(f"mv pdbs/{files[0]}/* pdbs")
  os.system(f"rm -r pdbs/{files[0]}")

print("[*] Installing dependencies")
# download and compile TM-Align
!wget https://zhanggroup.org/TM-align/TMalign.cpp
!g++ -static -O3 -ffast-math -lm -o TMalign TMalign.cpp

# install dependencies
!pip install -q umap-learn

Saving pdbs.zip to pdbs.zip
[*] Installing dependencies
--2022-07-19 01:13:48--  https://zhanggroup.org/TM-align/TMalign.cpp
Resolving zhanggroup.org (zhanggroup.org)... 141.213.137.249
Connecting to zhanggroup.org (zhanggroup.org)|141.213.137.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182097 (178K) [text/plain]
Saving to: ‘TMalign.cpp’

TMalign.cpp         100%[===================>] 177.83K   355KB/s    in 0.5s    

2022-07-19 01:13:50 (355 KB/s) - ‘TMalign.cpp’ saved [182097/182097]

     |████████████████████████████████| 88 kB 2.8 MB/s 
     |████████████████████████████████| 1.1 MB 26.6 MB/s 


In [ ]:
#@markdown ### Create the feature tensor.
use_TM_Align = True #@param {type:"boolean"}
#@markdown - Toggle whether pairwise TM-Align scores should be used as input features.
use_RMSD = True #@param {type:"boolean"}
#@markdown - Toggle whether pairwise RMSD scores should be used as input features.

if not any([use_TM_Align, use_RMSD]):
  from IPython.display import  display, HTML, IFrame
  display(HTML("<h1 style='color: #ff2950'>ERROR: At least one input feature must be selected.</h1>"))
  raise Exception()

import re, subprocess, os, json
from datetime import datetime
from multiprocessing import Pool
from itertools import combinations_with_replacement as cwr

### CONFIG
INPUT_FOLDER = "pdbs/"

def get_features(pdb_f1, pdb_f2):
  features = {}
  ## TM-Align Score
  output = subprocess.check_output(f"./TMalign {pdb_f1} {pdb_f2}", shell=True).decode()
  if use_RMSD:
    features["tma_score"] = float(re.findall(r"TM-score= ([.\d]*) ", output)[0])
    features["rmsd"] = float(re.search(r",\s*?RMSD=\s*?([.\d]*),", output, flags=re.DOTALL).group(1))

  return features


# Create a dictionary where protein identifiers are the keys and file paths are the values.
proteins = {}
for filename in os.listdir(INPUT_FOLDER):
  assert os.path.isfile(f"{INPUT_FOLDER}/{filename}") and filename[-4:] == ".pdb", f"{INPUT_FOLDER}/{filename} is NOT a valid pdb file."
  # TODO: Another exception if the alignment fails or if files are not pdb files.
  proteins[filename[:-4]] = f"{INPUT_FOLDER}/{filename}"

# get all possible combinations
combos = [tuple(sorted(x)) for x in cwr(proteins.keys(), 2)]
start = datetime.now()
print(f"[*] Scoring {len(combos)} structure combinations ({start})")

def wrapper(combo): return get_features(proteins[combo[0]], proteins[combo[1]])
# get TM-Align score of combinations
with Pool(1) as p:
  scores = p.map(wrapper, combos)

combos = {c:s for c,s in zip(combos,scores)}
end = datetime.now()
print(f"[*] Finished scoring at {end}, duration was ({(end-start).seconds}s)")

# create the matrix
matrix = {}
for t1 in proteins.keys():
  matrix[t1] = {}
  for t2 in proteins.keys():
    matrix[t1][t2] = combos[tuple(sorted([t1,t2]))]

# export to json
with open("tm-matrix.json", "w") as f:
  json.dump(matrix, f, indent=2)


# draw plots

[*] Scoring 2485 structure combinations (2022-07-19 01:14:29.827285)
[*] Finished scoring at 2022-07-19 01:33:24.279240, duration was (1134s)


In [ ]:
#@markdown ### Configure and view your output cluster.
cluster_algorithm = "UMAP" #@param ["UMAP", "t-SNE"]
#@markdown - **UMAP** is the default option as it's much faster and tends to preserve global structure better.
#@markdown - **t-SNE** is also available as an option and in some cases creates better projections than UMAP.
output_dimensions = 3 #@param ["2", "3"] {type:"raw"}
#@markdown - Choose a value for the output dimension (2D or 3D projection).

import json, os
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from umap import UMAP
import plotly.express as px

# read tm-score matrix file
df = pd.read_json("tm-matrix.json")

features = []
for row in df.to_numpy():
    features.append([list(x.values()) for x in row])
features = np.array(features)
features = features.reshape((features.shape[0], -1))

if cluster_algorithm == "UMAP":
  if output_dimensions == 2:
    proj = UMAP(n_components=2, init="random", random_state=0).fit_transform(features)
    fig = px.scatter(
      proj,
      x=0,
      y=1,
      title="Protein Clustering by Structural Similarity",
      labels={"color": "# Residues", "0":"x", "1":"y", "2":"z"},
      hover_name=df.columns,
      # text=df.columns,
    )
  else:
    proj = UMAP(n_components=3, init="random", random_state=0).fit_transform(features)
    fig = px.scatter_3d(
      proj,
      x=0,
      y=1,
      z=2,
      title="Protein Clustering by Structural Similarity",
      labels={"color": "# Residues", "0":"x", "1":"y", "2":"z"},
      hover_name=df.columns,
      # text=df.columns,
    )
else:
  if output_dimensions == 2:
    proj = TSNE(n_components=2, random_state=0).fit_transform(features)
    fig = px.scatter(
      proj,
      x=0,
      y=1,
      title="Protein Clustering by Structural Similarity",
      labels={"color": "# Residues", "0":"x", "1":"y", "2":"z"},
      hover_name=df.columns,
      # text=df.columns,
    )
  else:
    proj = TSNE(n_components=3, random_state=0).fit_transform(features)
    fig = px.scatter_3d(
      proj,
      x=0,
      y=1,
      z=2,
      title="Protein Clustering by Structural Similarity",
      labels={"color": "# Residues", "0":"x", "1":"y", "2":"z"},
      hover_name=df.columns,
      # text=df.columns,
    )

fig.update_layout(xaxis_title="", yaxis_title="")
fig.show()